# 从零开始实现

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import library.d2lzh_pytorch as d2l

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下,直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况,计算通道维上(axis=1)的均值和方差。这我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            # 训练模式下用当前的均值和方差标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1 - momentum) * mean
        moving_var = momentum * moving_var + (1 - momentum) * var
    Y = gamma * X_hat + beta # 拉伸和偏移
    return Y, moving_mean, moving_var

In [9]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数,分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量,全在内存上初始化成0
        self.moving_mean = nn.Parameter(torch.zeros(shape))
        self.moving_var = nn.Parameter(torch.zeros(shape))
    
    def forward(self, X):
        # 如果X不在内存上,将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用用.eval()后设成false
        Y, self.moving_mean, self.moving_var =  batch_norm(self.training,  X, self.gamma, self.beta, self.moving_mean,  self.moving_var, eps=1e-5, momentum=0.9)
        return Y

# LeNet 添加BatchNord

In [10]:
net = nn.Sequential(
    nn.Conv2d(1, 6, 5),
    BatchNorm(6, 4),
    nn.Sigmoid(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(6, 16, 5),
    BatchNorm(16, num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2, 2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4, 120),
    BatchNorm(120, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    BatchNorm(84, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [ ]:
batch_size = 256  
train_iter, test_iter =  d2l.load_data_fashion_mnist(batch_size=batch_size)  
lr, num_epochs = 0.001, 5  

optimizer = torch.optim.Adam(net.parameters(), lr=lr)  
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

# 快速实现

In [12]:
net = nn.Sequential(  
    nn.Conv2d(1, 6, 5), # in_channels, out_channels,  kernel_size  
    nn.BatchNorm2d(6),  
    nn.Sigmoid(),  
    nn.MaxPool2d(2, 2), # kernel_size, stride  
    nn.Conv2d(6, 16, 5),  
    nn.BatchNorm2d(16),  
    nn.Sigmoid(),  
    nn.MaxPool2d(2, 2),  
    d2l.FlattenLayer(),  
    nn.Linear(16*4*4, 120),  
    nn.BatchNorm1d(120),  
    nn.Sigmoid(),  
    nn.Linear(120, 84),  
    nn.BatchNorm1d(84),  
    nn.Sigmoid(),  
    nn.Linear(84, 10)  
)

In [13]:
batch_size = 256  
train_iter, test_iter =  d2l.load_data_fashion_mnist(batch_size=batch_size)  
lr, num_epochs = 0.001, 5  

optimizer = torch.optim.Adam(net.parameters(), lr=lr)  
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.0016, train acc 0.779, test acc 0.811,  time 7.0 sec
epoch 2, loss 0.2300, train acc 0.862, test acc 0.800,  time 7.7 sec
epoch 3, loss 0.1230, train acc 0.878, test acc 0.866,  time 7.9 sec
epoch 4, loss 0.0826, train acc 0.886, test acc 0.852,  time 7.0 sec
epoch 5, loss 0.0617, train acc 0.893, test acc 0.868,  time 7.1 sec
